# ipyleaflet: Interactive maps

## A Jupyter - LeafletJS bridge

## https://github.com/jupyter-widgets/ipyleaflet


ipyleaflet is a jupyter interactive widget library which provides interactive maps to the Jupyter notebook.

- MIT Licensed

**Installation:**

```bash
conda install -c conda-forge ipyleaflet
```

In [1]:
from __future__ import print_function

from ipyleaflet import (
    Map,
    Marker, MarkerCluster, TileLayer, ImageOverlay, GeoJSON,
    Polyline, Polygon, Rectangle, Circle, CircleMarker, Popup,
    SplitMapControl, WidgetControl,
    basemaps, basemap_to_tiles
)

from ipywidgets import HTML

In [2]:
center = [34.6252978589571, -77.34580993652344]
zoom = 10

# Map Widget

In [3]:
m = Map(center=center, zoom=zoom)
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [25]:
m.layout.height = '300px'
m.layout.width = '600px'
m.zoom = 6

### Every Layer in ipyleaflet is a Widget: TileLayers, Markers, Heatmaps, Videos...

You can dynamically change the layers you apply on top of the Map, for example you can dynamically change the base TileLayer and control it through a SelectionSlider widget:

In [26]:
from datetime import date, timedelta
from ipywidgets import SelectionSlider

n_days = 30
days = []

# Previous 30 days, starting from today
for n in range(n_days):
    delta = (n_days - n) * timedelta(days=1)
    days.append((date.today() - delta).isoformat())

days_slider = SelectionSlider(options=days, value=days[-1], description='Day: ')

base_layer = basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, days_slider.value)

m.layers = [base_layer]
m.zoom = 6


In [27]:
def handle_day_change(change, **kwargs):
    base_layer.url = basemaps.NASAGIBS.ModisTerraTrueColorCR.get('url') % change['new']

days_slider.observe(handle_day_change, 'value')

days_slider

SelectionSlider(description='Day: ', index=29, options=('2019-11-12', '2019-11-13', '2019-11-14', '2019-11-15'…

# jupyterlab-sidecar: A sidecar output widget for JupyterLab

## https://github.com/jupyter-widgets/jupyterlab-sidecar

In [28]:
from sidecar import Sidecar

sc = Sidecar(title='Map Output')
with sc:
    m.layout.height = ''
    display(m)

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

## Marker

In [29]:
mark = Marker(location=m.center)

In [30]:
m += mark

In [31]:
mark.visible

True

In [32]:
mark.visible = False

In [33]:
mark.visible = True

In [34]:
mark.interact(opacity=(0.0, 1.0, 0.01))

Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0, step=0.01),))

## Popup

The popup is displayed when you click on the marker. You can add ANY widget as a popup for the marker, from simple HTMLWidget to plots using bqplot.

In [35]:
from ipywidgets import FloatSlider, link

slider = FloatSlider(value=1.0, min=0.0, max=1.0)
link((mark, 'opacity'), (slider, 'value'))

mark.popup = slider

## Marker Cluster

Markers can be clustered depending on the zoom level.

In [36]:
xscale = 5
yscale = 10

x = [m.center[0] + i * xscale * .05 for i in (-1,0,1)]
y = [m.center[1] + i * yscale * .05 for i in (-1,0,1)]

from itertools import product
locations = product(x, y)
markers = [Marker(location=loc) for loc in locations]

The `MarkerCluster` will automatically handle clustering and the zoom level changes.

In [37]:
marker_cluster = MarkerCluster(markers=markers)
m += marker_cluster
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

## Heatmap

In [38]:
from ipywidgets import Button, IntSlider, link
from ipyleaflet import Heatmap
from random import gauss
import time

In [39]:
center = (37.09, -103.66)
zoom = 5

In [61]:
def create_random_data(length):
    "Return a list of some random lat/lon/value triples."
    return [[gauss(center[0], 2), 
             gauss(center[1], 4),
             gauss(700, 300)] for i in range(length)]

In [62]:
m.center = center
m.zoom = zoom

In [63]:
heat = Heatmap(locations=create_random_data(1000), radius=15, blur=10)
m.add_layer(heat)

In [64]:
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [65]:
def generate(_):
    heat.locations = create_random_data(1000)

button = Button(description='Generate data', button_style='success')
button.on_click(generate)
button

Button(button_style='success', description='Generate data', style=ButtonStyle())

In [66]:
slider = IntSlider(min=10, max=30, value=heat.radius)
link((slider, 'value'), (heat, 'radius'))
slider

IntSlider(value=15, max=30, min=10)

## Controls

In [67]:
# The following NASA images need a zoom level <= 9
if m.zoom > 9:
    m.zoom = 9

control = SplitMapControl(
    left_layer=basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, "2017-11-11") , 
    right_layer=basemap_to_tiles(basemaps.NASAGIBS.ModisAquaBands721CR, "2017-11-11")
)
m.add_control(control)

In [68]:
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [69]:
m.remove_control(control)

In [70]:
from ipywidgets import IntSlider, Button, link

button = Button(description='Goto NYC')

def goto_nyc(*args, **kwargs):
    # NYC: 40.7128° N, 74.0060° W
    m.center = (40.7128, -74.0060)
    m.zoom = 9

button.on_click(goto_nyc)

wid_control = WidgetControl(widget=button, position='bottomright')
m.add_control(wid_control)

## JupyterLab Themes Support

Controls in ipyleaflet have support for JupyterLab themes, try changing the current theme

In [71]:
m.remove_control(wid_control)

## Advanced example 1: Velocity

*It is ok to skip this for now -- running it requires downloading a ~1MB data file.*

This example needs an extra dependency that is not in the installation instructions and a dataset.

You can install it with this:

```bash
pip install netcdf4
```

To download the data, copy and paste the code below into a new cell and run it.

```python
import requests

wind_data = requests.get('https://github.com/benbovy/xvelmap/raw/master/notebooks/wind-global.nc')  

with open('wind-global.nc', 'wb') as f:  
    f.write(wind_data.content)
```


In [72]:
import requests

wind_data = requests.get('https://github.com/benbovy/xvelmap/raw/master/notebooks/wind-global.nc')  

with open('wind-global.nc', 'wb') as f:  
    f.write(wind_data.content)

In [73]:
from ipyleaflet import Velocity
import xarray as xr

In [74]:
center = (0, 0)
zoom = 4

m2 = Map(center=center, zoom=zoom, interpolation='nearest', basemap=basemaps.CartoDB.DarkMatter)

sc2 = Sidecar(title='Map Velocity')

with sc2:
    display(m2)

Map(basemap={'url': 'http://c.basemaps.cartocdn.com/dark_all/{z}/{x}/{y}.png', 'max_zoom': 20, 'attribution': …

In [75]:
ds = xr.open_dataset('wind-global.nc')
display_options = {
    'velocityType': 'Global Wind',
    'displayPosition': 'bottomleft',
    'displayEmptyString': 'No wind data'
}
wind = Velocity(data=ds,
                zonal_speed='u_wind',
                meridional_speed='v_wind',
                latitude_dimension='lat',
                longitude_dimension='lon',
                velocity_scale=0.01,
                max_velocity=20,
                display_options=display_options)
m2.add_layer(wind)
with sc2:
    display(m2)

Map(basemap={'url': 'http://c.basemaps.cartocdn.com/dark_all/{z}/{x}/{y}.png', 'max_zoom': 20, 'attribution': …

# Advanced example

In [76]:
from ipywidgets import Text, HTML, HBox
from ipyleaflet import GeoJSON, WidgetControl, Map 
import json

In [77]:
m = Map(center = (43,-100), zoom = 4)

geo_json_data = json.load(open('us-states-density-colored.json'))
geojson = GeoJSON(data=geo_json_data, hover_style={'color': 'black', 'dashArray': '5, 5', 'weight': 2})
m.add_layer(geojson)

html = HTML('''
    <h4>US population density</h4>
    Hover over a state
''')
html.layout.margin = '0px 20px 20px 20px'
control = WidgetControl(widget=html, position='topright')
m.add_control(control)

def update_html(properties, **kwargs):
    html.value = '''
        <h4>US population density</h4>
        <h2><b>{}</b></h2>
        {} people / mi^2
    '''.format(properties['name'], properties['density'])

geojson.on_hover(update_html)

m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

# Exercise

### 1) Create a Map

In [85]:
m = Map(center = (-40,-60), zoom=5)
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

### 2) Create a slider controlling the zoom level of the Map (manually, not using interact)

In [79]:
slider = IntSlider(min=3,max=7, step=1)
map_link = link((slider, 'value'), (m, 'zoom'))

slider

IntSlider(value=3, max=7, min=3)

### 3) Embed the slider in a WidgetControl on the bottom left of the map

In [80]:
widget_control = WidgetControl(widget=slider, position='bottomleft')
m.add_control(widget_control)